In [68]:
# noexport

!typech efficacy_utils.ipynb

[NbConvertApp] Converting notebook efficacy_utils.ipynb to python


In [11]:
from retention_utils import *

In [12]:
import jsonmemo as jsonmemo_module
jsonmemo_module.set_lowmem(True)

In [32]:
#installs_with_experiment_vars = get_installs_with_experiment_vars()

In [33]:
#print(installs_with_experiment_vars[0])

In [13]:


def get_condition_to_installs_for_random_assignment_abtest():
  abtest_name = 'difficulty_selection_screen'
  groups = ['survey_nochoice_nothing', 'survey_nochoice_easy', 'survey_nochoice_medium', 'survey_nochoice_hard']
  condition_to_installs = get_conditions_to_install_list_in_abtest_unstrict(abtest_name)
  for k in list(condition_to_installs.keys()):
    if k not in groups:
      del condition_to_installs[k]
  return condition_to_installs


In [35]:

#for install in installs_with_experiment_vars:
@msgpackmemo1arg
def get_domain_to_intervention_to_session_lengths_for_install(install):
  seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
  interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
  
  domain_to_session_to_intervention = {}
  for x in interventions_active_for_domain_and_session:
    if x.get('val') is None:
      continue
    interventions_active = json.loads(x['val'])
    if len(interventions_active) == 0:
      continue
    intervention_name = interventions_active[0]
    domain = x['key']
    session_id = x['key2']
    if domain not in domain_to_session_to_intervention:
      domain_to_session_to_intervention[domain] = {}
    domain_to_session_to_intervention[domain][session_id] = intervention_name
  
  domain_to_intervention_to_session_lengths = {}
  for x in seconds_on_domain_per_session:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_session')
      print(x)
      continue
    domain = x['key']
    session_id = x['key2']
    time_spent = x['val']
    if domain not in domain_to_session_to_intervention:
      continue
    intervention_name = domain_to_session_to_intervention[domain].get(session_id)
    if intervention_name is None:
      continue
    if domain not in domain_to_intervention_to_session_lengths:
      domain_to_intervention_to_session_lengths[domain] = {}
    if intervention_name not in domain_to_intervention_to_session_lengths[domain]:
      domain_to_intervention_to_session_lengths[domain][intervention_name] = []
    domain_to_intervention_to_session_lengths[domain][intervention_name].append(time_spent)
  return domain_to_intervention_to_session_lengths
  

In [52]:
# noexport

for x in get_collection_names():
  if 'per_day' in x and 'visits' not in x:
    print(x)
    break

edceda6041ef668213a96ba5_synced:seconds_on_domain_per_day


In [67]:
@msgpackmemo1arg
def get_epoch_to_domain_to_time_spent(install):
  seconds_on_domain_per_day = get_collection_for_install(install, 'synced:seconds_on_domain_per_day')
  output = {}
  for x in seconds_on_domain_per_day:
    if 'key' not in x:
      print('missing key in seconds_on_domain_per_day')
      print(x)
      continue
    domain = x['key']
    epoch = x['key2']
    seconds = x['val']
    if epoch not in output:
      output[epoch] = {}
    output[epoch][domain] = seconds
  return output

In [63]:
# noexport

for condition,installs in condition_to_installs.items():
  for install in installs:
    print(sorted(get_epoch_to_domain_to_time_spent(install).keys()))
    break

[1030, 1034, 1035, 1036, 1037, 1039, 1040, 1041, 1042, 1043, 1044, 1045, 1046, 1048, 1049, 1050, 1051, 1054, 1057, 1058, 1059, 1060, 1065, 1066, 1067, 1068, 1070, 1071, 1072, 1073, 1075, 1076, 1077, 1078, 1080, 1081, 1082, 1084, 1085, 1086, 1087, 1088, 1089, 1090, 1092, 1093, 1094, 1095, 1097, 1098, 1100, 1101, 1102, 1105, 1109, 1112, 1114, 1115, 1118, 1119, 1120, 1121, 1122, 1127, 1128, 1129, 1131, 1132, 1133, 1135, 1147, 1149, 1151, 1152, 1155, 1156, 1158, 1159, 1161, 1163, 1170, 1171, 1176, 1178, 1181, 1182, 1183, 1184, 1186, 1187, 1188, 1190, 1191, 1197, 1198, 1199, 1200, 1201, 1202, 1203, 1204, 1205, 1206, 1210, 1211, 1212, 1213, 1215, 1217, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1229, 1230, 1231, 1232, 1233, 1236, 1237, 1238, 1239, 1240]
[1067, 1068, 1069, 1070, 1071, 1072, 1073, 1074, 1075, 1076, 1077, 1079, 1080, 1081, 1082, 1086, 1087, 1096, 1097, 1098, 1099, 1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 11

In [37]:
# noexport

condition_to_installs = get_condition_to_installs_for_random_assignment_abtest()


In [178]:
import math

def make_domain_to_daily_time_dataframe():
  output = []
  for condition,installs in condition_to_installs.items():
    condition_to_lengths[condition] = []
    for install in installs:
      for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
        total_time_spent = sum(domain_to_time_spent.values())
        output.append({
          'user': install,
          'epoch': epoch,
          'time': total_time_spent,
          'logtime': math.log(total_time_spent),
          'condition': condition,
        })
#         for domain,total_time_spent in  domain_to_time_spent.items():
#           output.append({
#             'domain': domain,
#             'user': install,
#             'epoch': epoch,
#             'time': total_time_spent,
#             'logtime': math.log(total_time_spent),
#             'condition': condition,
#           })
  return to_dataframe(output)

In [179]:
df = make_domain_to_daily_time_dataframe()

In [138]:
#print(df)

In [180]:
%Rpush df

In [181]:
%%R
library(lme4)
#library(sjPlot)
library(lmerTest)
#library(stargazer)


In [ ]:
# %%R

# df$user <- as.factor(df$user)
# #df$domain <- as.factor(df$domain)
# df$condition <- as.factor(df$condition)
# df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
# df$epoch <- as.factor(df$epoch)
# df$logtime <- as.numeric(df$logtime)
# df$time <- as.numeric(df$time)
# summary(df)



In [177]:
%%R

df$user <- as.factor(df$user)
#df$domain <- as.factor(df$domain)
df$condition <- as.factor(df$condition)
df$condition <- factor(df$condition, levels = c("survey_nochoice_nothing", "survey_nochoice_easy", "survey_nochoice_medium", "survey_nochoice_hard"))
df$epoch <- as.factor(df$epoch)
df$logtime <- as.numeric(df$logtime)
df$time <- as.numeric(df$time)
summary(df)


#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user), data = df)
#results <- lmer(logtime ~ condition + (1|user) + (1|domain), data = df)
#show(results)
#show(summary(results))
#class(results) <- "lmerMod"
#stargazer(results)

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: logtime ~ condition + (1 | user)
   Data: df

REML criterion at convergence: 77301

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-6.3717 -0.3491  0.1870  0.5770  3.8005 

Random effects:
 Groups   Name        Variance Std.Dev.
 user     (Intercept) 1.547    1.244   
 Residual             1.906    1.381   
Number of obs: 21848, groups:  user, 385

Fixed effects:
                                 Estimate Std. Error        df t value Pr(>|t|)
(Intercept)                       7.95352    0.12913 288.51140  61.593   <2e-16
conditionsurvey_nochoice_easy    -0.08069    0.18473 277.37641  -0.437    0.663
conditionsurvey_nochoice_medium  -0.22269    0.18695 279.20254  -1.191    0.235
conditionsurvey_nochoice_hard    -0.29767    0.19554 282.93183  -1.522    0.129
                                   
(Intercept)                     ***
conditionsurvey_nochoice_easy      
conditionsurvey_no

In [199]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      condition_to_lengths[condition].append(total_time_spent)
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [46]:
# noexport
# bad


condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for domain,intervention_to_session_lengths in get_domain_to_intervention_to_session_lengths_for_install(install).items():
      if domain != 'www.youtube.com':
        continue
      for intervention,session_lengths in intervention_to_session_lengths.items():
        for session_length in session_lengths:
          condition_to_lengths[condition].append(session_length)


In [187]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    all_lengths_for_user = []
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      total_time_spent = sum(domain_to_time_spent.values())
      all_lengths_for_user.append(total_time_spent)
    if len(all_lengths_for_user) == 0:
      continue
    condition_to_lengths[condition].append(np.median(all_lengths_for_user))
      #if 'www.facebook.com' not in domain_to_time_spent:
      #  continue
      #time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      #condition_to_lengths[condition].append(time_on_facebook)

In [197]:
# noexport

condition_to_lengths = {}
for condition,installs in condition_to_installs.items():
  condition_to_lengths[condition] = []
  for install in installs:
    for epoch,domain_to_time_spent in get_epoch_to_domain_to_time_spent(install).items():
      if 'www.facebook.com' not in domain_to_time_spent:
        continue
      time_on_facebook = domain_to_time_spent.get('www.facebook.com', 0)
      condition_to_lengths[condition].append(time_on_facebook)

In [233]:
# noexport
from scipy.stats import ttest_ind

condition_to_mlengths = {}
for condition,session_lengths in condition_to_lengths.items():
  print(condition, np.median(session_lengths))
  condition_to_mlengths[condition] = np.median(session_lengths) / 60

print(ttest_ind(condition_to_lengths['survey_nochoice_hard'], condition_to_lengths['survey_nochoice_nothing']))
#print(ttest_ind(condition_to_lengths['survey_nochoice_easy'], condition_to_lengths['survey_nochoice_medium']))


survey_nochoice_medium 5643.0
survey_nochoice_nothing 6997.5
survey_nochoice_easy 4861.0
survey_nochoice_hard 4174.0
Ttest_indResult(statistic=-16.371668900032997, pvalue=1.513840323586665e-59)


In [243]:
from plot_utils import *

def plot_bar(label_with_value_list, **kwargs):
  print('plot_bar')
  labels = [x for x,y in label_with_value_list]
  remap_labels = kwargs.get('remap_labels')
  if remap_labels is not None:
    print(remap_labels)
    labels = [remap_labels.get(x, x) for x in labels]
  print('labels are')
  print(labels)
  data = [go.Bar(x=labels, y=[y for x,y in label_with_value_list])]
  plot_data(data, **kwargs)

def plot_dict_as_bar(d, **kwargs):
  data = []
  items = list(d.items())
  items.sort(key=lambda x: x[1], reverse=True)
  plot_bar(items, **kwargs)

In [244]:
#from plot_utils import *
plot_dict_as_bar(condition_to_mlengths, title='Total minutes spent daily on sites', xlabel='Difficulty', ylabel='Minutes spent daily on sites', remap_labels={'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard'})

plot_bar
{'survey_nochoice_nothing': 'Nothing', 'survey_nochoice_easy': 'Easy', 'survey_nochoice_medium': 'Medium', 'survey_nochoice_hard': 'Hard'}
labels are
['Nothing', 'Medium', 'Easy', 'Hard']


In [31]:
# for install in installs_with_experiment_vars:
#   #seconds_on_domain_per_session = get_collection_for_install(install, 'synced:seconds_on_domain_per_session')
#   #interventions_active_for_domain_and_session = get_collection_for_install(install, 'synced:interventions_active_for_domain_and_session')
#   res = get_domain_to_intervention_to_session_lengths_for_install(install)
#   #if len(res) > 0:
#   #  print(res)
#   #  break

exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack for arg 000a07dad34314af5a49d683
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/000a07dad34314af5a49d683.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0010f4ea92647449cfef843e.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0010f4ea92647449cfef843e.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0010f4ea92647449cfef843e.msgpack for arg 0010f4ea92647449cfef843e
done with computation 2019_05_

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0091af1d1d7fb9e1568c5dfd.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0092d78460333be953a75f34.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0092d78460333be953a75f34.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0092d78460333be953a75f34.msgpack for arg 0092d78460333be953a75f34
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0092d78460333be953a75f34.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/00a5afe4f698f6b525ba8ca3.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/00a5afe4f698f6b525ba8ca3.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0135f986e6ca92b5a1ac98e5.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0140097ba26bad1f796e5bf0.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0140097ba26bad1f796e5bf0.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0140097ba26bad1f796e5bf0.msgpack for arg 0140097ba26bad1f796e5bf0
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0140097ba26bad1f796e5bf0.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0144a500a0ce4d2d64e7e6f5.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0144a500a0ce4d2d64e7e6f5.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/019abdd547d520d09e60d410.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/01ace8319b196ce7b01dda4d.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/01ace8319b196ce7b01dda4d.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/01ace8319b196ce7b01dda4d.msgpack for arg 01ace8319b196ce7b01dda4d
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/01ace8319b196ce7b01dda4d.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/01af2a0f09bb9ad4b040c679.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/01af2a0f09bb9ad4b040c679.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/025045c26f930f0a8e0e0cf9.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/025045c26f930f0a8e0e0cf9.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/025045c26f930f0a8e0e0cf9.msgpack for arg 025045c26f930f0a8e0e0cf9
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/025045c26f930f0a8e0e0cf9.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0252312d35c1b257f4433e5c.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0252312d35c1b257f4433e5c.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/0252312d35c1b257f4433e5c.msgpack for arg 0252312d35c1b257f4433e5c
done with computation 2019_05_

...

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4cd882aa634f64f517b7f487.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4cd992646952f79a3c9193b6.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4cd992646952f79a3c9193b6.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4cd992646952f79a3c9193b6.msgpack for arg 4cd992646952f79a3c9193b6
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4cd992646952f79a3c9193b6.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4ce1340ae33be67616a89927.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4ce1340ae33be67616a89927.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d31602a3395e07c26f40c7c.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d473793588ccd8c1a21145c.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d473793588ccd8c1a21145c.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d473793588ccd8c1a21145c.msgpack for arg 4d473793588ccd8c1a21145c
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d473793588ccd8c1a21145c.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d4ce34ff6787f6683f60ac7.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4d4ce34ff6787f6683f60ac7.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4de09d5c03bfcaedf2e6a6ac.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4de192c901f85fb9257f7514.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4de192c901f85fb9257f7514.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4de192c901f85fb9257f7514.msgpack for arg 4de192c901f85fb9257f7514
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4de192c901f85fb9257f7514.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4dfd13e4c7f21713cfc7bd21.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4dfd13e4c7f21713cfc7bd21.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e4f6d5e09224de536cf23e2.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e5ba8178f910a1cec4bab29.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e5ba8178f910a1cec4bab29.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e5ba8178f910a1cec4bab29.msgpack for arg 4e5ba8178f910a1cec4bab29
done with computation 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e5ba8178f910a1cec4bab29.msgpack
exception in msgpackmemo1arg for file 2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e69dff69f4bdcfe49089814.msgpack
[Errno 2] 没有那个文件或目录: '2019_05_25/get_domain_to_intervention_to_session_lengths_for_install/4e69dff69f4bdcfe49089814.msgpack'
performing computation 2019_05_25/get_domain_to_intervention_to_

KeyboardInterrupt: 